# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [5]:
!pip install langchain-elasticsearch

   ---------------------------------------- 0.0/654.0 kB ? eta -:--:--
   ---------------------------------------- 654.0/654.0 kB 6.3 MB/s eta 0:00:00


In [3]:
!python3 -m pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass

Python was not found; run without arguments to install from the Microsoft Store, or disable this shortcut from Settings > Manage App Execution Aliases.


## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial. 

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [5]:
import os

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass.getpass("Elastic Cloud ID: ")
#ELASTIC_CLOUD_ID  = os.getenv('ELASTIC_CLOUD_ID')

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass.getpass("Elastic Api Key: ")
#ELASTIC_API_KEY  = os.getenv('ELASTIC_API_KEY')

# https://platform.openai.com/api-keys
OPENAI_API_KEY = getpass.getpass("OpenAI API key: ")
#OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

INDEX_NAME = "ind_nam"

vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name=INDEX_NAME, #give it a meaningful name,
    embedding=embeddings,
)

Elastic Cloud ID:  ········
Elastic Api Key:  ········
OpenAI API key:  ········


## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [7]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [89]:
!pip install jq

In [9]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


from datetime import datetime

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["name"] = record.get("name", "Unknown")
    metadata["summary"] = record.get("summary", "")
    metadata["url"] = record.get("url", "")
    metadata["category"] = record.get("category", "Uncategorized")

    # Handle empty 'updated_at' field
    updated_at = record.get("updated_at", "").strip()

    if updated_at:  # If it's not empty, keep it
        metadata["updated_at"] = updated_at
    else:  # If empty, set a default date (or remove it)
        metadata["updated_at"] = datetime.utcnow().isoformat()  # Example default: current UTC time

    return metadata



# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800, chunk_overlap=400 #define chunk size and chunk overlap
)
docs = loader.load_and_split(text_splitter=text_splitter)

C:\Users\Ignacio\AppData\Local\Temp\ipykernel_58256\2558737373.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  metadata["updated_at"] = datetime.utcnow().isoformat()  # Example default: current UTC time


### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [11]:
from elasticsearch.helpers import bulk
from elasticsearch import Elasticsearch

# Initialize Elasticsearch client
es_client = Elasticsearch(cloud_id=ELASTIC_CLOUD_ID, api_key=ELASTIC_API_KEY)

def index_documents(es, docs):
    actions = []
    for i, doc in enumerate(docs):
        try:
            action = {
                "_index": INDEX_NAME,
                "_id": str(i),
                "_source": {
                    "content": doc.page_content,
                    "vector": embeddings.embed_query(doc.page_content),  # Embeddings here
                    "metadata": doc.metadata,
                },
            }
            actions.append(action)
        except Exception as e:
            print(f"Error in document {i}: {e}")

    try:
        success, failed = bulk(es, actions, stats_only=False, raise_on_error=False)  # Capture all errors
        print(f"Successfully indexed {success} documents.")

        if failed:
            print(f"Failed to index {len(failed)} documents. Showing first 3 errors:")
            for error in failed[:3]:  # Print first 3 errors for debugging
                print(error)

    except Exception as e:
        print(f"Bulk indexing failed: {str(e)}")

# Run indexing
index_documents(es_client, docs)


Successfully indexed 15 documents.


In [13]:
documents = vectorstore.from_documents(
    docs,
    embeddings,
    index_name=INDEX_NAME,
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY,max_tokens=200)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [15]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible. 
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)


def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("what is the nasa sales team?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
The NASA sales team is a part of the Americas region in the sales organization of the company. It is responsible for promoting and selling the company's products and services in North and South America. The team is led by two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. They work closely with other departments, such as marketing, product development, and customer support, to ensure the company consistently delivers high-quality products and services to its clients.


**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?

In [17]:
ans2 = chain.invoke("How does NASA collaborate with private companies?")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the nature of collaboration between NASA and private companies?', '2. Can you explain the partnership between NASA and private companies?', '3. In what ways does NASA work together with private companies?']


In [19]:
print("---- Answer 2 ----")
print(ans2)

---- Answer 2 ----

NASA, which stands for North America South America region, is a part of the Americas region in the sales organization. It has two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. As a part of the sales team, NASA collaborates with private companies by identifying and pursuing new business opportunities, nurturing existing client relationships, and ensuring customer satisfaction. They also work closely with other departments, such as marketing, product development, and customer support, to deliver high-quality products and services to their clients. This collaboration helps NASA to effectively serve their customers and achieve their business objectives.


In [21]:
ans3 = chain.invoke("How much money does NASA use for their projects?")
print("\n---- Answer 3 ----")
print(ans3)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the budget allocated by NASA for their projects?', '2. How does NASA fund their projects financially?', '3. Can you provide information on the financial resources utilized by NASA for their projects?']



---- Answer 3 ----
I don't know the answer to that question as it is not mentioned in the retrieved context. The context only discusses the sales organization structure and objectives of a tech company, not the budget or funding of NASA.
